In [1]:
import tensorflow as tf

In [2]:
import numpy as np
import pandas as pd

In [3]:
n_inputs = 28 * 28
n_hidden1 = 300
n_hidden2 = 100
n_outputs = 10

In [4]:
X = tf.placeholder(tf.float32, shape=(None, n_inputs), name='X')

In [5]:
y = tf.placeholder(dtype=tf.int64, shape=(None), name='y')

In [6]:
from tensorflow.contrib.layers import fully_connected

In [7]:
hidden1 = fully_connected(X, n_hidden1, scope='hidden1')

In [8]:
hidden2 = fully_connected(hidden1, n_hidden2, scope='hidden2')

In [9]:
logits = fully_connected(hidden2, n_outputs, scope='outputs', activation_fn=None)

In [10]:
with tf.name_scope('loss'):
    xentropy = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=y, logits=logits)
    loss = tf.reduce_mean(xentropy)

In [11]:
with tf.name_scope('eval'):
    correct = tf.nn.in_top_k(logits, y, 1)
    accuracy = tf.reduce_mean(tf.cast(correct, tf.float32))

In [18]:
learning_rate = 0.01
with tf.name_scope('train'):
    optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate)
    training_op = optimizer.minimize(loss)

In [19]:
init = tf.global_variables_initializer()

In [13]:
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets('/tmp/data/')

Successfully downloaded train-images-idx3-ubyte.gz 9912422 bytes.
Extracting /tmp/data/train-images-idx3-ubyte.gz
Successfully downloaded train-labels-idx1-ubyte.gz 28881 bytes.
Extracting /tmp/data/train-labels-idx1-ubyte.gz
Successfully downloaded t10k-images-idx3-ubyte.gz 1648877 bytes.
Extracting /tmp/data/t10k-images-idx3-ubyte.gz
Successfully downloaded t10k-labels-idx1-ubyte.gz 4542 bytes.
Extracting /tmp/data/t10k-labels-idx1-ubyte.gz


In [15]:
with tf.variable_scope('', default_name='', reuse=True):
    weights1 = tf.get_variable("hidden1/weights")
    weights2 = tf.get_variable("hidden2/weights")

In [16]:
# clip weights
threshold = 1.0
clipped_weights1 = tf.clip_by_norm(weights1, clip_norm=threshold, axes=1)
clip_weights1 = tf.assign(weights1, clipped_weights1)
clipped_weights2 = tf.clip_by_norm(weights2, clip_norm=threshold, axes=1)
clip_weights2 = tf.assign(weights2, clipped_weights2)

In [17]:
n_epochs = 10
batch_size = 50

In [22]:
with tf.Session() as sess:
    init.run()
    for epoch in range(n_epochs):
        for iteration in range(mnist.train.num_examples // batch_size):
            X_batch, y_batch = mnist.train.next_batch(batch_size=batch_size)
            sess.run(training_op, feed_dict={X: X_batch, y: y_batch})
            clip_weights1.eval()
            clip_weights2.eval()
        acc_train = accuracy.eval(feed_dict={X: X_batch, y: y_batch})
        acc_test  = accuracy.eval(feed_dict={X: mnist.test.images, y: mnist.test.labels})
        print(epoch, "Train accuracy:", acc_train, "Test accuracy:", acc_test)

0 Train accuracy: 0.94 Test accuracy: 0.9006
1 Train accuracy: 0.9 Test accuracy: 0.9166
2 Train accuracy: 0.96 Test accuracy: 0.9289
3 Train accuracy: 0.94 Test accuracy: 0.9361
4 Train accuracy: 0.94 Test accuracy: 0.9406
5 Train accuracy: 0.96 Test accuracy: 0.9463
6 Train accuracy: 0.98 Test accuracy: 0.9495
7 Train accuracy: 0.96 Test accuracy: 0.9533
8 Train accuracy: 0.96 Test accuracy: 0.9568
9 Train accuracy: 0.98 Test accuracy: 0.9601
